In [39]:
import pandas as pd
import numpy as np
import os
import glob
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly
import datetime


In [52]:
pdf=pd.read_csv('./user_purchase.csv')
vdf=pd.read_csv('./user_visit.csv')

### Q1.

In [53]:
vdf['buy']=0
pdf['buy']=1

unique_pdf=pdf.drop_duplicates(subset='user_id')[['user_id','buy']]
merge_df=vdf.merge(unique_pdf,on='user_id',how='left')[['user_id','referrer','buy_y']]
merge_df.columns=['user_id','referrer','buy']
merge_df.fillna(0,inplace=True)
report_df=merge_df.groupby(['referrer','buy']).count()
report_df

user_id
referrer buy         
ad       0.0      353
         1.0       35
app      0.0      738
         1.0      259
search   0.0      339
         1.0      182

### Q2.

In [58]:
buy_vdf.groupby('referrer')['buy'].mean()

referrer
ad        0.090206
app       0.259779
search    0.349328
Name: buy, dtype: float64

### Q3.
- 各流入による、上位スタンプ
- 時間帯による、購入数
- 購入された商品の流入経路
- アクセスユーザの時間別流入

In [66]:
pdf['timestamp']=pd.to_datetime(pdf['timestamp'])
vdf['timestamp']=pd.to_datetime(vdf['timestamp'])
# pdf['minute']=pdf['timestamp'].dt.minute
# pdf['hour']=pdf['timestamp'].dt.hour

In [71]:

#集計するために、最初の要素から何時間経過したかを示す列を用意
start_time=pdf['timestamp'].min() if pdf['timestamp'].min()<vdf['timestamp'].min() else vdf['timestamp'].min()

pdf['delta_hour']=(pdf['timestamp']-start_time).apply(lambda x:x.total_seconds()/(60*60)).round()
vdf['delta_hour']=(vdf['timestamp']-start_time).apply(lambda x:x.total_seconds()/(60*60)).round()

pdf=pdf.dropna()
vdf=vdf.dropna()

#描画のために、delta_hourを時刻表示示する列を用意
pdf['about_time']=pdf['delta_hour'].apply(lambda x:start_time+datetime.timedelta(hours=int(x)))
vdf['about_time']=vdf['delta_hour'].apply(lambda x:start_time+datetime.timedelta(hours=int(x)))

In [72]:
item_time_df=pdf.groupby(['item_id','about_time'])['buy'].count().unstack(level='item_id').fillna(0).reset_index()
data=[plotly.graph_objs.Scatter(x=item_time_df['about_time'],y=item_time_df[referrer],name=referrer) for referrer in [f'item{str(i).zfill(2)}' for i in range(1,6)]]
fig=plotly.graph_objs.Figure(data=data)
fig.show()

In [73]:
pdf=pdf.merge(vdf[['user_id','referrer']],on='user_id',how='left')

In [79]:
#時間毎のアクセス数
user_count_df=vdf.groupby(['referrer','about_time'])['user_id'].count().unstack(level='referrer').fillna(0).reset_index()
data=[plotly.graph_objs.Scatter(x=user_count_df['about_time'],y=user_count_df[referrer],name=referrer) for referrer in ['ad','app','search']]
fig=plotly.graph_objs.Figure(data=data,title='時間あたりの閲覧数')
fig.show()

In [76]:
item_referrer_df=pdf.groupby(['item_id','referrer'])['buy'].count().fillna(0).reset_index()
fig=px.bar(item_referrer_df,x='item_id',y='buy',color='referrer',title='流入経路毎のitem購入数')
fig.update_xaxes(title_text='item_id')
fig.update_yaxes(title_text='購入数')

In [80]:
import plotly 
#1時間毎の売上を流入ごとに分けたデータ。
time_referrer_price_df=pdf.groupby(['referrer','about_time'])['price'].sum().unstack(level='referrer').fillna(0).reset_index()
data=[plotly.graph_objs.Scatter(x=time_referrer_price_df['about_time'],y=time_referrer_price_df[referrer],name=referrer) for referrer in ['ad','app','search']]
fig=plotly.graph_objs.Figure(data=data,title='時間毎のitem売上')
fig.show()

In [81]:
time_referrer_buy_df=pdf.groupby(['referrer','about_time'])['about_time'].count().unstack(level='referrer').fillna(0).reset_index()
time_referrer_visit_df=vdf.groupby(['referrer','about_time'])['about_time'].count().unstack(level='referrer').fillna(0).reset_index()



In [93]:
#時間毎の購入率
time_referrer_ratio_df=(time_referrer_buy_df.iloc[:,1:]/time_referrer_visit_df.iloc[:,1:]).fillna(0)
time_referrer_ratio_df['about_time']=time_referrer_buy_df['about_time']
data=[plotly.graph_objs.Scatter(x=time_referrer_ratio_df['about_time'],y=time_referrer_ratio_df[referrer],name=referrer) for referrer in ['ad','app','search']]
fig=plotly.graph_objs.Figure(data=data,title='時間毎の一人あたり購入数')
fig.show()